In [ ]:
import json
import string

import pandas as pd
import asyncio
from pathlib import Path

import spacy
from spacy.language import Language
from spacy.util import filter_spans

from openai import OpenAI, AsyncOpenAI
from dotenv import load_dotenv

load_dotenv()

In [14]:
ollama_model = "gemma2:9b"
openai_model = "gpt-4o-mini-2024-07-18"

In [15]:
openai_async_client = AsyncOpenAI()
ollama_client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama'
)

In [16]:
USER = 'user'
AI = 'assistant'
SYS = 'system'

In [17]:
def generate(client:OpenAI, model:str, messages:list[dict], stream:bool=True, output_json:bool=False):
    param = {
        'model':model,
        'messages':messages,
        'stream':stream
    }
    
    if output_json:
        param['response_format'] = {'type': 'json_object'}

    return client.chat.completions.create(**param)

def write_stream(stream) -> str:
    ret = ""
    for c in stream:
        dlt = c.choices[0].delta.content
        if dlt:
            ret += dlt
            print(dlt, end="", flush=True)

    return ret

def append_message(role:str, content:str, messages:list[dict]=[]):
    messages.append({'role': role, 'content': content})
    return messages

async def aget_response(client:AsyncOpenAI, model:str, messages:list[dict], output_json:bool=False) -> str:
    param = {
        'model':model,
        'messages':messages
    }
    if output_json:
        param['response_format'] = {'type': 'json_object'}

    ret = await client.chat.completions.create(**param)
    return ret.choices[0].message.content

In [18]:
trial_n = 3

In [19]:
original_text = """We will simulate the sales plan after the recall of the ZX-1000 model based on the following sales plan. Please fill in the blanks in the simulation according to the conditions.

<Reference 3: Sales Plan>
Scooter Model: ZX-1000
2023 Production Volume: 15,840.00 units
2024 Production Plan: 27,720.00 units
Domestic Inventory as of the end of April 2024: 3,960.00 units

Projected Revenue for This Fiscal Year: 2,772.00 million yen

(ZX-1000 Domestic Projected Revenue: 1,980.00 million yen)
</Reference 3: Sales Plan>

#Conditions
The recall cost per unit is set at 8,000 yen, which includes all costs such as parts, repairs, transportation, and other expenses.
The post-recall sales volume N is estimated with a reduction rate of 25%.

#Simulation
Let A be the production volume in 2023, B the production plan volume for 2024, C the inventory volume as of April 2024, D the planned revenue for this fiscal year, and E the planned revenue for this fiscal year for the model subject to recall.
The number of units sold subject to recall, NR: calculated by subtracting the number of units remaining unsold as of April 2024 from the 2023 production volume A, i.e., NR = A - C =
The sales price of the model subject to recall, P: calculated by dividing the planned sales revenue E of the model by the total of the production plan volume B and the inventory volume C for 2024, i.e., P = E / (B + C) ="""

In [20]:
list_mask_rate = [20, 40, 60, 80, 100]

In [23]:
# QA_TYPE = "UQA"
# QA_TYPE = "RQA"
QA_TYPE = "MskCal"

In [24]:
# Output paths
dir_output = Path(f"../data/output/{QA_TYPE}_2/")
if not dir_output.exists():
    dir_output.mkdir()

output_path_qa = dir_output / f"{QA_TYPE}_{ollama_model.replace(':', '_')}_{{trial_no}}.csv"
output_path_qa_tmp = dir_output / f"{QA_TYPE}_{ollama_model.replace(':', '_')}_tmp.csv" # 途中経過

output_path_step1 = dir_output / f"{QA_TYPE}_step1.csv"
output_path_step2 = dir_output / f"{QA_TYPE}_step2.csv"
output_path_step2_use = dir_output / f"{QA_TYPE}_step2_use.csv"
output_path_step4 = dir_output / f"{QA_TYPE}_step4.csv"

## step1

In [ ]:
@Language.component("merge_hyphenated")
def merge_hyphenated(doc):
    """Detect hyphenated words as a single word"""
    spans = []
    for i in range(len(doc) - 2):
        if doc[i + 1].text == '-' and not doc[i + 1].whitespace_:
            spans.append(doc[i:i+3])
    filtered_spans = filter_spans(spans)
    with doc.retokenize() as retokenizer:
        for span in filtered_spans:
            retokenizer.merge(span)
    return doc

def get_entity_token_index(doc:spacy.tokens.doc.Doc, start_i:int) -> list[int]:
    """Search until the end of the entity token and return the index up to the end position"""
    list_ind = [start_i]
    if len(doc) <= (start_i+1):
        return list_ind
    
    cur = start_i + 1
    while cur < len(doc):
        if doc[cur].ent_iob_ == "O":
            return list_ind
        
        list_ind.append(cur)
        cur += 1

    return list_ind

def process(nlp:spacy.language.Language, text:str) -> pd.DataFrame:
    """Output a list of morphemes from the text and return a dataframe of words (including duplicates)"""
    doc = nlp(text)

    list_words = []
    list_checked_i = []

    for token in doc:
        # entity token
        if token.ent_type_:
            # Start token of the entity
            if token.ent_iob_ == "B":
                tmp_index = get_entity_token_index(doc, token.i)
                tmp_words = doc[tmp_index[0]:tmp_index[-1]+1].text
                tmp_lemma = doc[tmp_index[0]:tmp_index[-1]+1].lemma_
                list_words.append(
                    {
                        'word': tmp_words,
                        'part_of_speech': token.pos_,
                        'category': token.ent_type_, 
                        'lemma': tmp_lemma,
                        'word_count': len(tmp_words.split()),
                        'index': tmp_index
                    }
                )
                list_checked_i.extend(tmp_index)
        else:
            word_data = {
                    'word': token.text,
                    'part_of_speech': token.pos_,
                    'category': token.ent_type_,
                    'lemma': token.lemma_,
                    'word_count': len(token.text.split()),
                    'index':[token.i]
                }
            
            if token.i in list_checked_i:
                continue
            
            if token.pos_ in ['PUNCT', 'SPACE']:
                word_data['word_count'] = 0

            list_words.append(
                word_data
            )
            list_checked_i.append(token.i)

    df = pd.DataFrame(list_words)

    return df

In [25]:
def step1(list_text:list) -> pd.DataFrame:
    """Extract morphemes from the text column of the QA"""
    list_result = []

    nlp = spacy.load("en_core_web_sm")
    nlp.add_pipe("merge_hyphenated", before='parser')

    for text in list_text:
        list_word_df = process(nlp, text)
        list_result.append(list_word_df)

    df_result = pd.concat(list_result)

    list_tmp = ['PUNCT', 'SPACE', 'AUX', 'CCONJ', 'SCONJ', 'ADP', 'PART', 'DET']
    df_result.loc[df_result['part_of_speech'].isin(list_tmp), 'word_count'] = 0

    list_tmp = ['CARDINAL', 'MONEY', 'PERCENT', 'DATE']
    df_result.loc[df_result['category'].isin(list_tmp), 'word_count'] = 0

    list_tmp = ['*', '=', '-', '+', '/', 'NR']
    df_result.loc[df_result['word'].isin(list_tmp), 'word_count'] = 0

    df_result.loc[df_result['word'].apply(len)==1, 'word_count'] = 0

    return df_result

In [18]:
df_word = step1([original_text])

## step2

In [26]:
STEP2_PROMPT = """The following meta table contains metadata extracted from Text according to the definition. Fill in the values for "Part of Speech", "Category", and "Meaning" in the table, and output the JSON in the format {{'data' : [ {{'word': str, 'part_of_speech': str, 'category': str, 'meaning': str}}, ...]}}.

## Meta Table
{step1_output}

## Definition
- Category: One of "organization name, individual name, technical term"
- Meaning: Words that express higher-level concepts (multiple possible). Do not use other "words".

## Example
Word | Part of Speech | Category | Meaning
---|---|---|---
Medical Team | Common Noun | Organization Name | Healthcare
Relay Station | Common Noun | Technical Term | Communication
Tanaka Vehicles | Proper Noun | Organization Name | Company, Manufacturing
Chronowar | Proper Noun | Technical Term | Product Name
Napoleon | Proper Noun | Individual Name | Historical Figure

## Text
{context_input}"""

In [27]:
def convert_df_to_markdown(df:pd.DataFrame):
    meta_table_s2 = ' | '.join(df.columns) + '\n'
    meta_table_s2 += ' | '.join(['---']*len(df.columns.to_list())) + "\n"
    meta_table_s2 += '\n'.join(df.apply(lambda x: ' | '.join([x[col] for col in df.columns]), axis=1).values)

    return meta_table_s2

In [28]:
def step2(client:OpenAI, model:str, df_word:pd.DataFrame, text:str) -> pd.DataFrame:
    """Create a word list at the lemma level from df_word and add category and meaning for content words"""
    list_result = []

    nlp = spacy.load("en_core_web_sm")
    nlp.add_pipe("merge_hyphenated", before='parser')

    # Group by lemma
    cols_unique = ['part_of_speech', 'category']
    df_gr_lemma = df_word.groupby('lemma').agg({col:'unique' for col in cols_unique}).reset_index().copy()
    for col in cols_unique:
        df_gr_lemma[col] = df_gr_lemma[col].apply(', '.join)
    df_gr_lemma = df_gr_lemma.rename(columns={'lemma':'word'})
    df_gr_lemma['meaning'] = ""
    df_gr_lemma['word_lower'] = df_gr_lemma['word'].apply(lambda x: x.lower().strip(string.punctuation))# 紐づけ用

    # Process 10 words at a time to avoid failure
    for start in range(0, len(df_gr_lemma), 10):
        end = start + 10
        chunk = df_gr_lemma.iloc[start:end]
        
        # Reprocess until the meaning is filled
        total_output_count = chunk.shape[0]
        list_tmp_result = []
        chunk_count = 10 
        while (total_output_count > 0) and (chunk_count > 0) :
            print(total_output_count, chunk['word'].unique())
            md_metatable = convert_df_to_markdown(chunk.drop(columns='word_lower'))

            messages = []
            prompt = STEP2_PROMPT.format(
                step1_output=md_metatable, 
                context_input=text
            )
            messages = append_message(USER, prompt, messages)
            
            # Generate
            count = trial_n
            chunk_count -= 1
            while count > 0:
                try:
                    response = generate(client, model, messages, stream=False, output_json=True).choices[0].message.content
                    data_s2 = json.loads(response)['data']
                    df_tmp = pd.DataFrame(data_s2)

                    # 1. Convert to lowercase and remove punctuation and join them
                    df_tmp_1 = df_tmp.copy()
                    df_tmp_1['word_lower'] = df_tmp_1['word'].apply(lambda x: x.lower().strip(string.punctuation)) 
                    df_tmp_1 = df_tmp_1[df_tmp_1['word_lower'].isin(chunk['word_lower'].unique())].copy()
                    df_tmp_1 = df_tmp_1[(df_tmp_1['meaning'].notna())&(df_tmp_1['meaning']!='')]
                    print(df_tmp_1['word'].unique())
                    list_tmp_result.append(df_tmp_1.drop(columns='word_lower').copy())
                    chunk = chunk[~chunk['word_lower'].isin(df_tmp_1['word_lower'].unique())]

                    # 2. Convert the extracted words to lemmas and join them
                    if chunk.shape[0] > 0:
                        df_tmp_2 = df_tmp.loc[~df_tmp.index.isin(df_tmp_1.index.to_list())].copy()
                        df_tmp_2['lemma_'] = df_tmp_2['word']
                        for i, row in df_tmp_2.iterrows():
                            doc = nlp(row['word'])
                            df_tmp.loc[i, 'lemma_'] = doc[0:].lemma_

                        # Join
                        df_tmp_2 = df_tmp_2[df_tmp_2['lemma_'].isin(chunk['word'])]
                        print(df_tmp_2['lemma_'].unique())
                        list_tmp_result.append(df_tmp_2.drop(columns='lemma_').copy())
                        chunk = chunk[~chunk['word'].isin(df_tmp_2['lemma_'].unique())]

                    total_output_count = chunk.shape[0]
                    break
                
                except Exception as e:
                    print(': error', e)
                    count -= 1
            
        list_result.extend(list_tmp_result)

    return pd.concat(list_result, ignore_index=True)

In [ ]:
df_lemma = step2(ollama_client, ollama_model, df_word[df_word['word_count']!=0], original_text)

In [26]:
df_word.to_csv(output_path_step1, encoding='utf-8-sig', index=False)
df_lemma.to_csv(output_path_step2, encoding='utf-8-sig', index=False)

#### Create a word list for code conversion

In [30]:
df_word_use = df_word[df_word['word_count']!=0].copy()
df_word_use['code'] = df_word_use.groupby('lemma').ngroup()
df_word_use['code'] = df_word_use['code'].rank(method='dense').astype(int)
df_word_use['code'] = df_word_use['code'].apply(lambda x: "r"+str(x).zfill(3))

In [30]:
# Merging process
list_merge = []
checked_index = []
usecols = ['word', 'part_of_speech', 'category', 'lemma', 'word_count', 'index', 'code',
       'part_of_speech_output', 'category_output', 'meaning']
# 1. Merge the lemma and word columns
df_merge_tmp = df_word_use.merge(df_lemma.reset_index(), how='inner', left_on=['lemma'], right_on=['word'], suffixes=['', '_output'])
list_merge.append(df_merge_tmp[usecols].fillna('').copy())
df_lemma_tmp = df_lemma.loc[~df_lemma.index.isin(df_merge_tmp['index_output'])].copy()[df_lemma.columns]
checked_index.extend(df_merge_tmp.index.tolist())

In [31]:
#  2. Convert all to lowercase and merge
df_word_use_tmp = df_word_use[~df_word_use.index.isin(checked_index)].copy()
df_word_use_tmp['word_lower'] = df_word_use_tmp['lemma'].apply(lambda x: x.lower().strip(string.punctuation))
df_lemma_tmp['word_lower'] = df_lemma_tmp['word'].apply(lambda x: x.lower().strip(string.punctuation))
df_merge_tmp = df_word_use_tmp.merge(df_lemma_tmp.drop(columns=['word']).reset_index(), how='inner', on='word_lower', suffixes=['', '_output'])
list_merge.append(df_merge_tmp[usecols].fillna('').copy())
df_lemma_tmp = df_lemma_tmp.loc[~df_lemma_tmp.index.isin(df_merge_tmp['index_output'])].copy()[df_lemma.columns]
checked_index.extend(df_merge_tmp.index.tolist())

In [32]:
# 3. Reconvert the word in df_lemma to a lemma and merge
df_word_use_tmp = df_word_use[~df_word_use.index.isin(checked_index)].copy()
df_word_use_tmp['word_lower'] = df_word_use_tmp['lemma'].apply(lambda x: x.lower().strip(string.punctuation))
df_lemma_tmp['lemma'] = None

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("merge_hyphenated", before='parser')

for i, row in df_lemma_tmp.iterrows():
    doc = nlp(row['word'])
    df_lemma_tmp.loc[i, 'lemma'] = doc[0:].lemma_

df_lemma_tmp['word_lower'] = df_lemma_tmp['lemma'].apply(lambda x: x.lower().strip(string.punctuation))

df_merge_tmp = df_word_use_tmp.merge(df_lemma_tmp.drop(columns=['word']).reset_index(), how='inner', on='word_lower', suffixes=['', '_output'])
list_merge.append(df_merge_tmp[usecols].fillna('').copy())
df_lemma_tmp = df_lemma_tmp.loc[~df_lemma_tmp.index.isin(df_merge_tmp['index_output'])].copy()[df_lemma.columns]
checked_index.extend(df_merge_tmp.index.tolist())

In [33]:
df_word_meaning = pd.concat(list_merge + [df_word_use.loc[~df_word_use.index.isin(checked_index)]], ignore_index=True)

for col in ['part_of_speech', 'category']:
    df_word_meaning[f"{col}_new"] = df_word_meaning[f'{col}']
    df_word_meaning.loc[df_word_meaning[f"{col}_new"]=="", f"{col}_new"] = df_word_meaning[df_word_meaning[f"{col}_new"]==""][f'{col}_output']

df_word_new = df_word_meaning[['lemma', 'word', 'part_of_speech_new', 'category_new', 'meaning', 'code']].rename(columns=
    {
        'part_of_speech_new': 'part_of_speech',    
        'category_new': 'category',    
    }
)

In [35]:
df_word_new.to_csv(output_path_step2_use, encoding='utf-8-sig', index=False)

## step4
- Select words to be coded based on the masking rate

In [31]:
import random
seed = 12345

random.seed(seed)

In [32]:
def select_mask_row(df_word:pd.DataFrame, mask_rate:int):
    """Select words to be coded based on the masking rate"""
    assert (mask_rate >= 0) and (mask_rate <= 100), f"`mask_rate` must be set between 0 and 100. mask_rate: {mask_rate}"

    df_copy = df_word.copy()
    mask_col = f'p_{mask_rate}_masked'
    df_copy[mask_col] = False

    list_lemma = df_word['lemma'].unique().tolist()
    mask_row_n = round(len(list_lemma) * (mask_rate/100))
    print(f'{mask_rate}% number of words to be masked:', mask_row_n)

    list_mask_lemma = random.sample(list_lemma, k=mask_row_n)
    mask_rows_index = df_word[df_word['lemma'].isin(list_mask_lemma)].index.values

    df_copy.loc[mask_rows_index, mask_col] = True

    return df_copy

In [ ]:
for mask_rate in list_mask_rate:
    df_word_new = select_mask_row(df_word_new, mask_rate)

In [33]:
def step4(text:str, list_mask_rate:list[int], df_word_new:pd.DataFrame) -> pd.DataFrame:
    """Return masked text based on the masking rate"""
    df_copy = pd.DataFrame()
    for mask_rate in list_mask_rate:
        df_copy[f"s4_prg_encode_{mask_rate}"] = ""

        values = df_word_new[df_word_new[f'p_{mask_rate}_masked']].apply(lambda x: {x['code']:x['word']}, axis=1)
        values = sorted(values, key=lambda x: x[next(iter(x))].count(' '), reverse=True)

        sub_text = text
        for pair in values:
            code = list(pair.keys())[0]
            word = list(pair.values())[0]
            sub_text = sub_text.replace(word, f"<{code}>")
        df_copy.loc[0, f"s4_prg_encode_{mask_rate}"] = sub_text

    return df_copy

In [41]:
df_qa = step4(original_text, list_mask_rate, df_word_new)

In [43]:
df_qa.to_csv(output_path_qa_tmp, encoding='utf-8-sig', index=False)

In [44]:
df_word_new.to_csv(output_path_step4, encoding='utf-8-sig', index=False)

In [34]:
df_word_new = pd.read_csv(output_path_step4)

In [ ]:
# Calculate the missing rate of meanings for each masking rate
list_tmp = []
for mask_rate in list_mask_rate:
    if mask_rate == 0:
        continue
    
    tmp_total_lemma = df_word_new[df_word_new[f'p_{mask_rate}_masked']]['lemma'].nunique()
    tmp_empty_meaning_lemma = tmp_total_lemma - df_word_new[df_word_new[f'p_{mask_rate}_masked'] & df_word_new['meaning'].notna()]['lemma'].nunique()
    list_tmp.append({'MR': mask_rate, 'number of words with missing meanings': tmp_empty_meaning_lemma, 'word count': tmp_total_lemma, 'missing rate of meanings': tmp_empty_meaning_lemma / tmp_total_lemma})
    
pd.DataFrame(list_tmp)

## step5
- generate answers

In [35]:
import asyncio

In [36]:
STEP5_PROMPT = """{text}
Therefore, the total recall cost X is, X = 8,000 * NR =
Since the planned sales volume is B + C, considering the reduction rate, the post-recall sales volume N is, N = (B + C) * (1 - 0.25) =
The decrease in revenue Y is, Y = P * (B + C) * 0.25 =
The loss amount L is, L = X + Y =
The revised planned sales revenue for the model subject to recall, E', is, E' = E - L =
The revised planned revenue for this fiscal year, D', is, D' = D - L =

<Meta Information>
{metadata}
</Meta Information>"""

In [37]:
async def exec_step5(client:AsyncOpenAI, model:str, i:int, output_col:str, query_col:str, text:str, metadata: str, count:int):
    messages = []
    prompt = STEP5_PROMPT.format(
        text=text,
        metadata=metadata
    )
    messages = append_message(USER, prompt, messages)

    while count > 0:
        try:
            res = await aget_response(client, model, messages)
            ret = (i, res, prompt, output_col, query_col)
            return ret

        except Exception as e:
            print('error', e)
            count -= 1

    return (i, '-1', '', output_col, query_col)

In [38]:
async def step5(aclient:AsyncOpenAI, model:str, df_qa:pd.DataFrame, df_word_new:pd.DataFrame) -> pd.DataFrame:
    df_copy = df_qa.copy()
    print('model:', model)

    results = []

    for mask_rate in list_mask_rate:
        tasks = []
        
        output_col = f'answer_{model}_{mask_rate}'
        query_col = f'query_{mask_rate}'
        df_copy[output_col] = ""
        df_copy[query_col] = ""
        
        col_q = f's4_prg_encode_{mask_rate}'

        for i, row in df_copy.iterrows():
            meta_table = df_word_new[df_word_new[f'p_{mask_rate}_masked']].groupby('lemma').agg(
                {col: 'unique' for col in ['part_of_speech', 'category', 'meaning', 'code']}
            ).copy()
            if not meta_table.empty:
                for col in ['part_of_speech', 'category', 'meaning', 'code']:
                    meta_table[col] = meta_table[col].apply(lambda x: ', '.join(t for t in x if type(t)==str))
            md_metatable = convert_df_to_markdown(meta_table)
            count = trial_n
            tasks.append(asyncio.ensure_future(exec_step5(aclient, model, i, output_col, query_col, row[col_q], md_metatable, count)))

        results.extend(await asyncio.gather(*tasks))

    for i, ans, query, output_col, query_col in results:
        df_copy.loc[i, output_col] = ans
        df_copy.loc[i, query_col] = query

    return df_copy.copy()

In [ ]:
for output_num in range(10):
    print('trial:', output_num) 
    df_result = await step5(openai_async_client, openai_model, df_qa, df_word_new)
    save_path_tmp = str(output_path_qa).format(trial_no=output_num)
    print(save_path_tmp)
    df_result.to_csv(save_path_tmp, encoding='utf-8-sig', index=False)

## GPT-4o

In [39]:
openai_model = "gpt-4o-2024-08-06"

In [44]:
output_path_qa = dir_output / f"{QA_TYPE}_{openai_model.replace(':', '_')}_{{trial_no}}.csv"

In [45]:
df_qa = pd.read_csv(output_path_qa_tmp, encoding='utf-8-sig')

In [ ]:
for output_num in range(10):
    print('trial:', output_num) 
    df_result = await step5(openai_async_client, openai_model, df_qa, df_word_new)
    save_path_tmp = str(output_path_qa).format(trial_no=output_num)
    print(save_path_tmp)
    df_result.to_csv(save_path_tmp, encoding='utf-8-sig', index=False)